<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/ComfyUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚠️ **IMPORTANT: LINEAR EXECUTION MODE**

**Please read before running:**

1.  **Strict Linear Flow:** This notebook is designed to be run strictly in order (Cell 1 -> Cell 2 -> Cell 3).
2.  **Define Models First:** You **must** define your model download URLs in **Cell 2** (The Downloader) *before* you launch the server in Cell 3.
3.  **How to Add/Change Models:**
    * You cannot download models via the ComfyUI browser interface efficiently in this mode.
    * To add a model: **STOP** the running Cell 3 (The Launcher), add the new URL to **Cell 2**, run Cell 2 again, and then **RE-RUN** Cell 3.

In [ ]:
#@title Install ComfyUI and Dependencies

import os

# 1. Define Paths
project_path = "/content/ComfyUI"
manager_path = "/content/ComfyUI/custom_nodes/ComfyUI-Manager"

# 2. Clone ComfyUI if not present
if not os.path.exists(project_path):
    print("🚀 Cloning ComfyUI...")
    !git clone https://github.com/comfyanonymous/ComfyUI $project_path
else:
    print("✅ ComfyUI already cloned.")

# 3. Install Cloudflared (Tunnel)
if not os.path.exists("/usr/local/bin/cloudflared"):
    print("🌐 Installing Cloudflared...")
    !wget -q -O /content/cloudflared-linux-amd64.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i /content/cloudflared-linux-amd64.deb
    !rm /content/cloudflared-linux-amd64.deb

# 4. Install Dependencies (Quietly to save log space)
print("📦 Installing Python Dependencies (Torch, Xformers, etc.)...")
%cd $project_path
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --no-cache-dir

# 5. Install ComfyUI Manager
if not os.path.exists(manager_path):
    print("🔧 Installing ComfyUI Manager...")
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git $manager_path
else:
    print("✅ ComfyUI Manager installed.")

print("\n🎉 Environment Ready. Proceed to Step 2.")

In [ ]:
# ==============================================================================
# 📥 MODEL DOWNLOADER
# Paste your Direct Download Links below using the format:
# !wget -c "URL" -P "TARGET_PATH"
# -c allows resuming interrupted downloads.
# ==============================================================================

!wget -c "https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors" -P "/content/ComfyUI/models/text_encoders/"

print("✅ Downloads processing complete.")

In [ ]:
#@title Launch ComfyUI
import subprocess
import threading
import time
import socket
import os

# ==============================================================================
# 🧹 ZOMBIE PROCESS CLEANUP
# This section ensures port 8188 is freed if you restarted the cell.
# ==============================================================================
print("🧹 Cleaning up zombie processes and releasing Port 8188...")
!pkill -f main.py
!pkill -f cloudflared
!fuser -k 8188/tcp 2>/dev/null
time.sleep(2) # Give the OS a moment to release the port

# ==============================================================================
# 🌐 START CLOUDFLARED TUNNEL
# ==============================================================================
# We start this in the background, but print the URL before blocking.
print("🚀 Launching Cloudflared Tunnel...")
tunnel_log = "/content/cloudflared.log"
# Start cloudflared in the background
subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://localhost:8188"],
    stdout=open(tunnel_log, "w"),
    stderr=subprocess.STDOUT
)

# Wait for the URL to generate
found_url = False
attempt = 0
print("⏳ Waiting for Tunnel URL...")
while not found_url and attempt < 20:
    if os.path.exists(tunnel_log):
        with open(tunnel_log, "r") as f:
            for line in f:
                if "trycloudflare.com" in line:
                    import re
                    url = re.search(r'https://.*\.trycloudflare\.com', line)
                    if url:
                        print(f"\n🔗 \033[92mYOUR COMFLUI URL: {url.group(0)}\033[0m \n")
                        found_url = True
                        break
    time.sleep(2)
    attempt += 1

if not found_url:
    print("⚠️ Could not extract Cloudflare URL. Check /content/cloudflared.log")

# ==============================================================================
# 🎨 START COMFLYUI (BLOCKING)
# ==============================================================================
print("🔥 Starting ComfyUI... (This cell will block execution, do not close tab)")
%cd /content/ComfyUI
# We run this directly so the output streams to the cell
!python main.py --listen --port 8188